In [1]:
## fix seeds 
SEED = 42

import random as rn
rn.seed(SEED)

import numpy as np
np.random.seed(SEED)

import tensorflow as tf
tf.random.set_seed(SEED)

In [2]:
## Define BigEarthNet dataset object
import sys
sys.path.append("..")

from src.dataset.big_earth_dataset import BigEarthDataset
big_earth = BigEarthDataset('/workspace/data/prepped/dev-train.tfrecord')
train_dataset = big_earth.dataset
val_dataset = BigEarthDataset('/workspace/data/prepped/dev-test.tfrecord', shuffle_buffer_size=0).dataset

In [3]:
from src.model.resnet50 import ResNet50
from codecarbon import EmissionsTracker

tracker = EmissionsTracker()

classifier = ResNet50(input_shape = (120, 120, 10), classes = 5)
classifier.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[
        'accuracy', 
        tf.keras.metrics.AUC(),
        tf.keras.metrics.Precision(),
        tf.keras.metrics.Recall()
    ],
    
)

tracker.start()
classifier.fit(
    train_dataset, 
    class_weight=big_earth.class_weights,
    epochs=2,
    validation_data=val_dataset,
    callbacks=[
        tf.keras.callbacks.CSVLogger('logs/metrics.csv'), 
        tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5), 
        tf.keras.callbacks.ReduceLROnPlateau(), 
        tf.keras.callbacks.TensorBoard('logs/test', profile_batch=0)  
    ],
    workers=4,
    use_multiprocessing=True
)
emissions = tracker.stop()
print(emissions)

[08/12/2020 21:32:56] [   DEBUG] [codecarbon.core.cpu] [is_powergadget_available():27] [PID:14 TID:139850564859712] CODECARBON : Exception occurred while instantiating IntelPowerGadget : CODECARBON : Platform not supported by Intel Power Gadget
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/codecarbon/core/cpu.py", line 22, in is_powergadget_available
    IntelPowerGadget()
  File "/usr/local/lib/python3.6/dist-packages/codecarbon/core/cpu.py", line 61, in __init__
    self._setup_cli()
  File "/usr/local/lib/python3.6/dist-packages/codecarbon/core/cpu.py", line 89, in _setup_cli
    "CODECARBON : Platform not supported by Intel Power Gadget"
SystemError: CODECARBON : Platform not supported by Intel Power Gadget
[08/12/2020 21:32:56] [   DEBUG] [codecarbon.core.cpu] [is_rapl_available():39] [PID:14 TID:139850564859712] CODECARBON : Exception occurred while instantiating RAPLInterface : CODECARBON : Intel RAPL files not found at /sys/class/powercap/int

/usr/local/lib/python3.6/dist-packages/tzlocal/unix.py:158: UserWarning: Can not find any timezone configuration, defaulting to UTC.
  warnings.warn('Can not find any timezone configuration, defaulting to UTC.')


[08/12/2020 21:32:59] [    INFO] [apscheduler.scheduler] [_real_add_job():881] [PID:14 TID:139850564859712] Added job "BaseEmissionsTracker._measure_power" to job store "default"
[08/12/2020 21:32:59] [    INFO] [apscheduler.scheduler] [start():166] [PID:14 TID:139850564859712] Scheduler started
[08/12/2020 21:32:59] [   DEBUG] [apscheduler.scheduler] [_process_jobs():940] [PID:14 TID:139847993898752] Looking for jobs to run
[08/12/2020 21:32:59] [   DEBUG] [apscheduler.scheduler] [_process_jobs():1020] [PID:14 TID:139847993898752] Next wakeup is due at 2020-12-08 21:33:11.616239+00:00 (in 12.061524 seconds)
Epoch 1/2
[08/12/2020 21:33:11] [   DEBUG] [apscheduler.scheduler] [_process_jobs():940] [PID:14 TID:139847993898752] Looking for jobs to run
[08/12/2020 21:33:11] [   DEBUG] [apscheduler.scheduler] [_process_jobs():1020] [PID:14 TID:139847993898752] Next wakeup is due at 2020-12-08 21:33:26.616239+00:00 (in 14.967282 seconds)
[08/12/2020 21:33:11] [    INFO] [apscheduler.executors

In [4]:
results = classifier.evaluate(val_dataset)
print("Loss :", results[0])
print("Accuracy :", results[1])
classifier.predict(val_dataset)

1/1 [==============================] - 0s 10ms/step - loss: 23889.8379 - accuracy: 0.0526 - auc: 0.5729 - precision: 0.4912 - recall: 0.6829
Loss : 23889.837890625
Accuracy : 0.05263157933950424


ModuleNotFoundError: No module named 'codecarbon'